# Video Capturing

In [2]:
import numpy as np
import cv2 as cv
import PIL as ImageGrab
import pyautogui as py
import win32api, win32con 
import keyboard
import time

# Image Processing

In [5]:
def takeSS():
    im = py.screenshot('image.png',region=(5,30, 847, 542))
    # Change the format that OpenCV can understand
    img = np.array(im)
    img_cvt = cv.cvtColor(img, cv.COLOR_RGB2BGR)

    # cv.imshow("das", img_cvt)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return img_cvt

Edge detection using Canny Edge Detection 

In [6]:
def canny_edge_detection(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY )
    kernel = 5
    blur = cv.GaussianBlur(gray, (kernel, kernel),0)
    canny = cv.Canny(blur, 50, 150)
    
    return canny

# frame = takeSS()
# canny_image = canny_edge_detection(frame)

# cv.imwrite('canny_edge_result.jpg',canny_image)

# cv.imshow("win",canny_image)
# cv.waitKey(0)
# cv.destroyAllWindows()

Specify the region of interest

In [17]:
def roi(img):
    height = img.shape[0]
    width = img.shape[1]
    
    # Road mask
    mask = np.zeros_like(img)
    car_mask = np.zeros_like(img)
    
    # Define the coordinates for the trapezoid region
    # Adjust these points based on the shape of the lane and the car's position
    roi_points = np.array([
    [((250), (250)),    # Top-left corner
     ((600), (250)),    # Top-right corner     
     ((800), (300)),    # Bottom-right corner
     ((50),  (300))]    # Bottom-left corner    
    ], np.int32)

     # HARDCODE
    # triangle = np.array([
    # car_triangle = np.array([
    #     [(int(width * 0.1), height),  # Bottom-left point
    #      (int(width * 0.5), int(height * 0.5)),  # Top-middle point
    #      (int(width * 0.9), height)]  # Bottom-right point
    # ], np.int32)

    # Fill the triangular region
    cv.fillPoly(mask, roi_points, 255)
    #cv.fillPoly(mask, [car_triangle],0)
    
    # Use bitwise_and to apply the mask
    masked_image = cv.bitwise_and(img, mask)
    
    cv.imshow("ROI Applied", masked_image)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    return masked_image

# Example usage:
# img_cvt = takeSS()  # Capture screenshot and convert
# edges = canny_edge_detection(img_cvt)  # Perform Canny edge detection
# masked_image = roi(edges)

# cv.imshow("Masked Image", masked_image)
# cv.waitKey(0)
# cv.destroyAllWindows()


# For future adjustment, use trapezoid instead of triangle

In [18]:
def draw_road_roi(img):
    height, width = img.shape[0], img.shape[1]

    # Define a triangle that captures the road area in front of the car
    # triangle = np.array([
    #     [(int(width * 0.1), height),           # Bottom-left: 10% from the left of the frame
    #      (int(width * 0.5), int(height * 0.6)), # Top-middle: center of the frame, 60% up
    #      (int(width * 1.2), height)]           # Bottom-right: 90% from the left of the frame
    # ], np.int32)
    # HARDCODE
    roi_points = np.array([
    [((250), (250)),    # Top-left corner
     ((600), (250)),    # Top-right corner     
     ((800), (300)),    # Bottom-right corner
     ((50),  (300))]    # Bottom-left corner    
    ], np.int32)



    # Draw the triangle on the image for visualization
    cv.polylines(img, [roi_points], isClosed=True, color=(0, 255, 0), thickness=2)
    return img

# Example usage:
img_cvt = takeSS()  # Capture screenshot and convert
img_with_triangle = draw_road_roi(img_cvt)

cv.imshow("Road Roi", img_with_triangle)
cv.waitKey(0)
cv.destroyAllWindows()


Coordinate Explanation:
* (int(width * 0.1), height) defines the bottom-left point of the triangle (10% of the frame width from the left, and at the bottom).
* (int(width * 0.5), int(height * 0.5)) is the top-middle point of the triangle (center of the width, and 50% down from the top).
* (int(width * 0.9), height) is the bottom-right point of the triangle (90% of the frame width from the left, and at the bottom).
How to Adjust the Triangle:
Visually Inspect the Lane:

Run the program and see where the triangle is placed. 
1. Adjust the width * 0.1 and width * 0.9 values to move the triangle left or right.
2. Adjust height * 0.5 to raise or lower the top point of the triangle.

Define the HLD (Hough Line Detection)

In [32]:
# Detect lines using Hough Transform
def houghlines(img):
    # Use HoughLinesP to detect line segments (parameters: rho, theta, threshold, min line length, max line gap)
    hough_lines = cv.HoughLinesP(img, 2, np.pi / 180, 110, np.array([]), minLineLength=40, maxLineGap=10)
    return hough_lines

The houghlines function detects line segments in a binary image using the Probabilistic Hough Line Transform and returns a list of detected line segments. This is often used in applications like lane detection where you need to find and analyze straight lines in an image.

In [33]:
# Create points from slope and intercept to form the line on the image
def make_points(img, lineSI):
    slope, intercept = lineSI  # Extract the slope and intercept of the line
    height = img.shape[0]
    y1 = int(height)  # y1 is the bottom of the image
    y2 = int(y1 * 1.5)  # Extend the line slightly above the top of the image for visualization
    
    # Calculate the corresponding x1, x2 points based on the slope and intercept
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)

    return [[x1, x2, y1, y2]]  # Return the calculated points as a list

# Before do this below code, you need to make sure that the line of the lane and the car
# is already correct. Specify the correct region of ineterest first
# Calculate the average slope and intercept for detected lines
def average_slope_intercepts(img, lines):
    left_fit = []  # To store the lines on the left (negative slope)
    right_fit = []  # To store the lines on the right (positive slope)
    
    # Iterate through each detected line
    for line in lines:
        for x1, x2, y1, y2 in line:
            # Fit a linear equation (y = mx + b) to get the slope (m) and intercept (b)
            fit = np.polyfit((x1, x2), (y1, y2), 1)
            slope = fit[0]
            intercept = fit[1]

            # Check if the slope is negative or positive (left or right lane)
            if slope < 0:
                left_fit.append((slope, intercept))  # Negative slope for the left lane
            else:
                right_fit.append((slope, intercept))  # Positive slope for the right lane

    # Calculate the average slope and intercept for left and right lines
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)

    # Create lines based on the average slope and intercept
    left_line = make_points(img, left_fit_average)
    right_line = make_points(img, right_fit_average)
    
    # Return the average left and right lines
    average_lines = [left_line, right_line]
    return average_lines

# Draw the lines on the image
def display_lines_average(img, lines):
    line_image = np.zeros_like(img)  # Create an empty image to draw lines on
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                # Draw the lines (red color, thickness = 10)
                cv.line(img, (x1, y1), (x2, y2), (0, 0, 255), 10)
    return img


# Run the lane detection
frame = takeSS()  # Capture a screenshot of the game frame
canny_image = canny_edge_detection(frame)  # Apply Canny edge detection to find edges
m_img = roi(canny_image)  # Apply the region of interest mask
h_lines = houghlines(m_img)  # Detect lines using Hough transform

# Calculate the average slope and intercept for the left and right lane lines
avg_lines = average_slope_intercepts(frame, h_lines)
print(avg_lines)  # Print the average line parameters
print(h_lines)  # Print the raw detected lines

# Draw the detected lines on the image
lines_image = display_lines_average(frame, avg_lines)
cv.imshow("Detected Lines", lines_image)  # Display the image with the detected lines
cv.waitKey(0)
cv.destroyAllWindows()

[[[-473, -1186, 542, 813]], [[422, 612, 542, 813]]]
[[[528 250 732 300]]

 [[132 300 322 250]]

 [[185 300 341 250]]

 [[544 260 673 300]]

 [[505 250 661 300]]

 [[210 261 258 250]]

 [[149 275 271 253]]]


def average_slope_intercepts(img, lines):

sometimes(in most cases actually) it detects multiple lines.  to overcome this, we can use the average value within the lines as you’ve seen in this code.

bear in minds that, the lines (you can see at the ros function) is a lot of them. So you need to specify that only certain lines that is captured, hence it is called region of interest.

# Controlling the car

In [22]:
import time
import pyautogui as py
import pydirectinput

# Function to click at specified coordinates (using PyAutoGUI)
def click(x, y):
    py.click(x, y)  # Click at the specified coordinates
    time.sleep(0.5)  # Wait briefly to ensure the game is focused
    
def accelerate(hold_time):
    start = time.time()
    while time.time() - start < hold_time:
        pydirectinput.keyDown('up')
    pydirectinput.keyUp('up')

def turnLeft(hold_time):
    start = time.time()
    while time.time() - start < hold_time:
        pydirectinput.keyDown('a')
    pydirectinput.keyUp('a')

def turnRight(hold_time):
    start = time.time()
    while time.time() - start < hold_time:
        pydirectinput.keyDown('d')
    pydirectinput.keyUp('d')

# Accelerate for 10 seconds, then turn left and right
click(409, 275)
accelerate(10)
turnLeft(0.3)
turnRight(0.3)

In [20]:
py.position()

Point(x=409, y=275)